In [1]:
from tqdm import tqdm
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from matplotlib import pyplot as plt
device = torch.device("cuda:1")

## Data pre-processing ##

In [2]:
# Load raw data
base_info = pd.read_csv('EnergyContest/BSinfo.csv')
cell_data = pd.read_csv('EnergyContest/CLdata.csv', parse_dates=['Time'])
energy_data = pd.read_csv('EnergyContest/ECdata.csv', parse_dates=['Time'])
power_consumption_prediction = pd.read_csv('EnergyContest/power_consumption_prediction.csv', parse_dates=['Time'])

# Combine CLdata and BSinfo
Total = pd.merge(base_info,cell_data, on=['BS', 'CellName'], how='inner')
Total_dummies = pd.get_dummies(Total, columns=['Antennas','Bandwidth','Frequency']) # one-hot encoding
Total_pivot = pd.pivot_table(Total_dummies, 
                             values=[col for col in Total_dummies.columns if (col != 'BS') and (col != 'Time') and (col != 'Mode') and (col != 'RUType')], 
                             index=['Time', 'BS', 'Mode', 'RUType'], 
                             columns='CellName', 
                             fill_value=0)
Total_pivot.reset_index(inplace=True)
Total_pivot.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in Total_pivot.columns]
Total_pivot = pd.get_dummies(Total_pivot, columns=['Mode','RUType'])

# Add feature : mean of load, sum of load, mean of ESMode, sum of ESMode
load_columns = [col for col in Total_pivot.columns if col.startswith('load')]
Total_pivot['load_avg'] = Total_pivot[load_columns].mean(axis=1)
Total_pivot['load_sum'] = Total_pivot[load_columns].sum(axis=1)
ESMode_columns = [col for col in Total_pivot.columns if col.startswith('ESMode')]
Total_pivot['ESMode_avg'] = Total_pivot[ESMode_columns].mean(axis=1)
Total_pivot['ESMode_sum'] = Total_pivot[ESMode_columns].sum(axis=1)

# Combine features and labels
train_data = pd.merge(energy_data,Total_pivot, on=['Time', 'BS'], how='left')
test_data = pd.merge(power_consumption_prediction, Total_pivot, on=['Time', 'BS'], how='left')
combined_data = pd.concat([train_data, test_data], ignore_index=False)


# perform one-hot encoding on time and BS
combined_data['Time'] = pd.to_datetime(combined_data['Time']).dt.dayofyear*24+pd.to_datetime(combined_data['Time']).dt.hour 
combined_data = pd.get_dummies(combined_data, columns=['Time','BS'])

# Obtain training set and test set
train_data = combined_data.iloc[:len(train_data)]
test_data = combined_data.iloc[len(train_data):]

# Delete features with a standard deviation of 0
def preprocess_data(df, columns_with_std_zero):
    X = df.drop(columns=['Energy','w'] + columns_with_std_zero)
    y = df['Energy']
    return X, y

columns_with_std_zero = train_data.columns[train_data.std() == 0].tolist()

# divide training set and test set into features and labels
X,y = preprocess_data(train_data,columns_with_std_zero)
X_test,y_test = preprocess_data(test_data,columns_with_std_zero)
print(X.columns)
print(train_data.columns)
print(y)

Index(['Antennas_1_Cell0', 'Antennas_1_Cell1', 'Antennas_2_Cell0',
       'Antennas_2_Cell1', 'Antennas_2_Cell2', 'Antennas_2_Cell3',
       'Antennas_32_Cell0', 'Antennas_4_Cell0', 'Antennas_4_Cell1',
       'Antennas_64_Cell0',
       ...
       'BS_B_990', 'BS_B_991', 'BS_B_992', 'BS_B_993', 'BS_B_994', 'BS_B_995',
       'BS_B_996', 'BS_B_997', 'BS_B_998', 'BS_B_999'],
      dtype='object', length=1163)
Index(['Energy', 'Antennas_1_Cell0', 'Antennas_1_Cell1', 'Antennas_1_Cell2',
       'Antennas_1_Cell3', 'Antennas_2_Cell0', 'Antennas_2_Cell1',
       'Antennas_2_Cell2', 'Antennas_2_Cell3', 'Antennas_32_Cell0',
       ...
       'BS_B_990', 'BS_B_991', 'BS_B_992', 'BS_B_993', 'BS_B_994', 'BS_B_995',
       'BS_B_996', 'BS_B_997', 'BS_B_998', 'BS_B_999'],
      dtype='object', length=1320)
0        64.275037
1        55.904335
2        57.698057
3        55.156951
4        56.053812
           ...    
92624    14.648729
92625    14.648729
92626    13.452915
92627    13.602392
92628 

In [3]:
# Define loss function
def MAPELoss(output, target):
    return torch.mean(torch.abs((target - output) / target))

def calculate_mape(y_true, y_pred):
    y_true = y_true.cpu().detach().numpy()
    y_pred = y_pred.cpu().detach().numpy()
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_wmape(y_true,y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

def WMAPELoss(y_pred, y_true, weights):
    return torch.sum(weights * torch.abs(y_true - y_pred)) / torch.sum(weights)

# Define Model
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.fc(x)
        return x * y

class LinearSEBlock(nn.Module):
    def __init__(self, input_dim, output_dim, reduction=16, activation="leakyrelu", use_bn=True, use_se=True):
        super(LinearSEBlock, self).__init__()
        
        # Linear block
        self.fc = nn.Linear(input_dim, output_dim)
        if use_bn:
            self.bn = nn.BatchNorm1d(output_dim)
        else:
            self.bn = None
        
        if activation == "leakyrelu":
            self.act = nn.LeakyReLU()
        elif activation == "relu":
            self.act = nn.ReLU()
        
        # SE layer
        if use_se:
            self.se = SELayer(output_dim, reduction)
        else:
            self.se = None

    def forward(self, x):
        x = self.fc(x)
        if self.bn:
            x = self.bn(x)
        x = self.act(x)
        if self.se:
            x = self.se(x)
        return x

class MyModel_attention(nn.Module):
    def __init__(self, input_dim, layers_dims=[512, 256, 128, 64], use_se_layers=[], **kwargs):
        super(MyModel_attention, self).__init__()
        dims = [input_dim] + layers_dims

        self.blocks = nn.ModuleList()
        
        if 0 in use_se_layers:
            se_kwargs = {key: kwargs[key] for key in kwargs if key not in ['activation', 'use_bn']}
            self.input_se = SELayer(input_dim, **se_kwargs)
            use_se_layers.remove(0)  
        else:
            self.input_se = None

        for i in range(len(dims)-1):
            self.blocks.append(LinearSEBlock(dims[i], dims[i+1], use_se=(i+1 in use_se_layers), **kwargs))
        
        self.fc_final = nn.Linear(layers_dims[-1], 1)
        
    def forward(self, x):
        if self.input_se:
            x = self.input_se(x)
        for block in self.blocks:
            x = block(x)
        x = self.fc_final(x)
        return x
        
def weight_init(m):
    if isinstance(m, torch.nn.Conv1d):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
    elif isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

## Train ##

In [4]:
# Splitting the dataset into training and testing sets
X_tensor = torch.Tensor(X.values).to(device)
y_tensor = torch.Tensor(y.values).unsqueeze(1).to(device)
X_data = TensorDataset(X_tensor, y_tensor)
X_loader = DataLoader(dataset=X_data, batch_size=2**12, shuffle=True)
X_test_tensor = torch.Tensor(X_test.values).to(device)
y_test_tensor = torch.Tensor(y_test.values).unsqueeze(1).to(device)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_data, batch_size=8192, shuffle=False)

# Define the parameters of the model
continue_train = 0
input_dim = X.shape[1]
layers_dims = [1024, 512, 256, 128, 64]
use_se_layers = []
reduction = 16
activation = "leakyrelu"
use_bn = True
num_epochs = 6000
filename = f"./model_TestBS-in-TrainingSet.pth"

In [5]:
import random

# Set random seed
seed = 42
random.seed(seed)  
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Set model
if continue_train == 1:
    model = torch.load(filename)

else:
    model = MyModel_attention(input_dim=input_dim, layers_dims=layers_dims, use_se_layers=use_se_layers, reduction=reduction, activation=activation, use_bn=use_bn).to(device)
    # model.apply(weight_init)

# Define loss function and optimizer
criterion = MAPELoss
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=3000, gamma=0.1)  

# Train the model
progress_bar_info = []
pbar = tqdm(range(num_epochs),desc="Processing", ncols=150)

try:
    for epoch in pbar:
        start_time = time.time()
        running_loss = 0.0
        running_mape = 0.0
        model.train()  # Set the model to training mode
        
        for i, data in enumerate(X_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # calculate running loss and MAPE
            running_loss += loss.item()*100

        scheduler.step()

        end_time = time.time()
        epoch_duration = end_time - start_time  # Calculate the duration of each epoch

        postfix_str = "Loss: {:.4f}, Epoch Duration: {:.2f} seconds".format(running_loss / (i+1), epoch_duration) # i+1 == len(X_loader)
        progress_bar_info.append(postfix_str)
        pbar.set_postfix_str(postfix_str)

        if  running_loss / len(X_loader) <= 0.23:
            break
        
    torch.save(model, filename)
    print(f"Model saved to {filename}")
    print('Finished Training')

except KeyboardInterrupt:
    torch.save(model, filename)
    print(f"Model saved to {filename}")

# 创建进度信息文件的文件名
progress_filename = filename.replace(".pth", "_progress.txt")
with open(f"./{progress_filename}", "w") as f:
    for line in progress_bar_info:
        f.write(line + "\n")


Processing:   0%|                                                                                                            | 0/6000 [00:00<?, ?it/s]

Processing:  50%|██████████████████████████▋                          | 3024/6000 [59:25<58:28,  1.18s/it, Loss: 0.2205, Epoch Duration: 1.16 seconds]

Model saved to ./model_TestBS-in-TrainingSet.pth
Finished Training


## Evaluate ##

In [5]:
evaluate_flag = 0
if evaluate_flag == 1:
    model = torch.load("BestModel/model_best_TestBS-in-TrainingSet.pth")
else:
    model = torch.load("./model_TestBS-in-TrainingSet.pth")
model.eval()  # Set the model to evaluation mode
predictions = []
with torch.no_grad(): 
    for i, data in enumerate(test_loader, 0):
        inputs, _ = data  # Ignore the weights
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())

predictions_numpy = np.array(predictions)
predictions_flatten = predictions_numpy.flatten()
predictions_series = pd.Series(predictions_flatten, name="PredictedEnergy")

test_data_new = pd.read_csv('EnergyContest/power_consumption_prediction.csv')
test_data_new['Energy'] = predictions_series
test_data_new['ID'] = test_data_new['Time'].astype(str) + "_" + test_data_new['BS'].astype(str)
test_data_new = test_data_new[['ID', 'Energy']]
test_data_new.to_csv('./power_consumption_prediction_TestBS-in-TrainingSet.csv', index=False)